In [0]:
%pip install python-dotenv

In [0]:
import os
import time
import requests

from pyspark.sql import functions as F
from pyspark.sql.types import StringType, ArrayType, DoubleType
from delta.tables import DeltaTable

from dotenv import load_dotenv

from PIL import Image, ImageDraw
import numpy as np

import geopandas as gpd

In [0]:
IMAGE_SIZE = [512, 512]
catalog_dev = "`land_topografisk-gdb_dev`"
schema_dev = "ai2025"
silver_table = f"{catalog_dev}.{schema_dev}.polygons_silver"

In [0]:
load_dotenv()
brukerid = os.getenv("GEONORGE_BRUKERID")
passord = os.getenv("GEONORGE_PASSORD")

def get_token():
    url = (
        f"https://baat.geonorge.no/skbaatts/req?brukerid={brukerid}"
        f"&passord={passord}&tjenesteid=wms.nib&retformat=s"
    )
    raw_token = requests.get(url).text.strip("`")
    return raw_token

token = get_token()
token_start_time = time.time()
token_lifetime = 55 * 60  # sekunder

def refresh_token_if_needed():
    global token, token_start_time
    if time.time() - token_start_time > token_lifetime:
        print("🔄 Fornyer token...")
        token = get_token()
        token_start_time = time.time()

In [0]:
# Er noe feil her

def generate_binary_mask(geom, out_path, width=512, height=512):
    """
    Lager en binær maske basert på geometrien (Shapely Polygon) og lagrer den som PNG.
    Polygonen tegnes med verdi 1 (hvit), og bakgrunn er 0 (svart).
    """
    try:
        minx, miny, maxx, maxy = geom.bounds

        mask = Image.new("L", (width, height), 0)
        draw = ImageDraw.Draw(mask)

        def world_to_pixel(x, y):
            px = int((x - minx) / (maxx - minx) * width)
            py = int((maxy - y) / (maxy - miny) * height)  
            return (px, py)

        exterior_coords = [world_to_pixel(x, y) for x, y in np.array(geom.exterior.coords)]
        draw.polygon(exterior_coords, outline=1, fill=1)

        mask.save(out_path)
        return True

    except Exception as e:
        print(f"❌ Feil ved maskegenerering: {e}")
        return False

In [0]:
def download_image(url: str, out_path: str) -> bool:
    try:
        r = requests.get(url, timeout=10)
        if r.status_code == 200:
            with open(out_path, "wb") as f:
                f.write(r.content)
            return True
    except Exception as e:
        print(f"❌ Feil ved nedlasting: {e}")
    return False

In [0]:
dom_output_dir = "/Volumes/land_topografisk-gdb_dev/external_dev/static_data/DL_SNUPLASSER/dom/"
image_output_dir = "/Volumes/land_topografisk-gdb_dev/external_dev/static_data/DL_SNUPLASSER/img/"
mask_dir = "/Volumes/land_topografisk-gdb_dev/external_dev/static_data/DL_SNUPLASSER/lab/"
os.makedirs(dom_output_dir, exist_ok=True)
os.makedirs(image_output_dir, exist_ok=True)
os.makedirs(mask_dir, exist_ok=True)


In [0]:
silver_df = spark.read.table(silver_table)

# LAST NED DOM
pending_dom = spark.read.table(silver_table).filter("dom_status = 'PENDING'").select("row_hash", "dom_path").collect()
for row in pending_dom:
    row_hash, dom_url = row["row_hash"], row["dom_path"]
    out_path = f"{dom_output_dir}/dom_{row_hash}.png"
    if download_image(dom_url, out_path):
        DeltaTable.forName(spark, silver_table).update(
            condition=f"row_hash = '{row_hash}'",
            set={"dom_status": "'DOWNLOADED'"}
        )

# Last ned ortofoto
pending_img = spark.read.table(silver_table).filter("image_status = 'PENDING' AND dom_status = 'DOWNLOADED'").select("row_hash", "image_path").collect()
for row in pending_img:
    refresh_token_if_needed()
    row_hash, image_base = row["row_hash"], row["image_path"]
    if image_base and image_base.startswith("http"):
        image_url = f"{image_base}{token}"
        out_path = f"{image_output_dir}/image_{row_hash}.png"
        if download_image(image_url, out_path):
            DeltaTable.forName(spark, silver_table).update(
                condition=f"row_hash = '{row_hash}'",
                set={"image_status": "'DOWNLOADED'"}
            )



gdf = gpd.GeoDataFrame(df.toPandas(), geometry="Polygons", crs="EPSG:25833")

for _, row in gdf.iterrows():
    out_path = f"{mask_dir}/mask_{row['row_hash']}.png"
    success = generate_binary_mask(row["Polygons"], out_path)

    if success:
        DeltaTable.forName(spark, silver_table).update(
            condition=f"row_hash = '{row['row_hash']}'",
            set={"mask_status": "'GENERATED'"}
        )
    else:
        print(f"⚠️ Maskegenerering feilet for {row['row_hash']}")